In [1]:
import time

In [2]:
import sde

In [3]:
def generateText(length):
    BASIC_TEXT = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas facilisis mauris urna, et porttitor velit efficitur suscipit. Phasellus id gravida augue. Duis erat odio, elementum ac mollis ac, scelerisque in nunc. Vestibulum finibus id ante a ultricies. Morbi at facilisis neque. Aliquam vel lorem purus. Morbi scelerisque mauris ut dolor dapibus pretium. Aliquam quis diam turpis. Nunc consectetur sapien magna. Phasellus fermentum et lorem ac dictum. Fusce molestie blandit eleifend. Suspendisse maximus est sit amet neque suscipit volutpat."
    return (BASIC_TEXT*(length // len(BASIC_TEXT) + 1))[:length]

In [4]:
assert(len(generateText(12345)) == 12345)
print(generateText(20))

Lorem ipsum dolor si


In [5]:
def testWithoutSDE(people, length):
    text = generateText(length)

    starttime = time.time()
    memory = 0

    for i in range(people):
        # Generate password for person N
        dataKey = sde.AESEncrypter.GeneratePassword()

        # Encrypt text by their password
        dataEncrypter = sde.AESEncrypter(dataKey)
        encryptedText = dataEncrypter.encryptString(text.encode())

        memory += len(dataKey)
        memory += len(encryptedText)

    totaltime = time.time() - starttime

    return totaltime, memory

In [6]:
def testWithSDE(people, length):
    text = generateText(length)

    starttime = time.time()
    memory = 0

    dataKey = sde.AESEncrypter.GeneratePassword()

    memory += len(dataKey)
    
    dataEncrypter = sde.AESEncrypter(dataKey)
    encryptedText = dataEncrypter.encryptString(text.encode())

    memory += len(encryptedText)
    
    for i in range(people):
        # Generate encrypter for person N
        userEncrypter = sde.RSAEncrypter()
        encryptedDataKey = userEncrypter.encryptString(dataKey)
        publicKey = userEncrypter.getEncodedPublicKey()
        privateKey = userEncrypter.getEncodedPrivateKey()

        memory += len(encryptedDataKey)
        memory += len(publicKey)
        memory += len(privateKey)
    
    totaltime = time.time() - starttime

    return totaltime, memory

In [7]:
def comparePerformance(people, length):
    withoutSDE = testWithoutSDE(people, length)
    withSDE = testWithSDE(people, length)
    print('Share {} bytes with {} people'.format(length, people))
    print('        {:>10} {:>10}'.format('time', 'memory'))
    print('without {:>10.5f} {:>10}'.format(withoutSDE[0], withoutSDE[1]))
    print('with    {:>10.5f} {:>10}'.format(withSDE[0], withSDE[1]))

In [8]:
comparePerformance(2, 10000000)

Share 10000000 bytes with 2 people
              time     memory
without    0.15506   20000064
with       0.23252   10006570


In [9]:
comparePerformance(100, 1000000)

Share 1000000 bytes with 100 people
              time     memory
without    4.49067  100003200
with       6.96685    1327070


In [10]:
comparePerformance(100, 3500)

Share 3500 bytes with 100 people
              time     memory
without    4.35475     353200
with       7.73770     330546
